In [11]:
import openai
import re,os
# os.environ["HTTP_PROXY"] = "http://127.0.0.1:7890"
# os.environ["HTTPS_PROXY"] = "http://127.0.0.1:7890"

openai.api_key = "sk-8p38chfjXbbL1RT943B051229a224a8cBdE1B53b5e2c04E2"
openai.api_base = "https://api.ai-yyds.com/v1"


class NPC():
    def __init__(self, name, desc, model="chatglm_6b",temperature=0.1,top_p=0.2):
        # model
        self.model = model
        self.temperature = temperature
        self.top_p = top_p
        # NPC
        self.name = name
        self.desc = desc
        self.memeory = []
        self.location = "星露谷小镇村口的石头广场，每天中午有玩耍的孩子，现在有2个小孩;广场上有茶桌，麻将桌；天上有只会飞的大象"
        self.mood = "平静"
        self.memeory.extend([
            {"role": "system", "content": rf"""
            请你扮演角色'{self.name}', 人物特点为'{self.desc}',心情是'{self.mood}',下面扮演该角色进行符合口语交流习惯的对话，仅生成单轮对话，不要添加多轮对话。仅生成角色嘴说出的话，不输出环境描述；当对话应该结束的时候，仅仅输出"<EOC>"即可。不要添加特殊符号，如：换行符、空格、制表符、转义符等。
            """},
                             ])

    def append_memory(self, memory):
        self.memeory.append(memory)

    def listen(self, content, npc):
        content = re.sub(r'(\\)+("|\'|\\)', '', content)
        response_template = rf"{content}<对方{npc.name}在‘{npc.location}’说话>"
        self.memeory.append({"role": "user", "content": response_template})

    def say(self):
        assert self.memeory[-1]["role"] == "user", rf"{self.name}:请先让对方NPC说话"
        try:
            response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                                  messages=self.memeory)
            words = response["choices"][0]["message"]["content"].strip()
            words = re.sub(r'(\\)+("|\'|\\)', '', words)
            self.memeory.append({"role": "assistant", "content": words})
            return words
        except Exception as e:
            import traceback
            traceback.print_exc()
            print(f"Error occurred: {e}")
            return "对不起，我现在无法回答你的问题。"


# 创建两个NPC
male = NPC(name="李大爷", desc="李大爷住在村口的小木屋#1里，每天中午都喜欢出来喝茶，平常喜欢下棋打麻将，并且非常古板喜欢讽刺。李大爷曾经偷过村长的一只黄金西瓜，但是村长并不知道。", model="chatglm_6b")
female = NPC(name="王大妈", desc="王大妈住在村口的小木屋#2里，每天中午都喜欢出来煮茶，平常喜欢看麻将，讨厌下棋，并且喜欢较真，是村长的老婆。在寻找黄金西瓜。", model="chatglm_6b")

# 添加种子对话;大妈先跟大爷说话
seed = "中午好啊!我的黄金西瓜不见了，你见过吗"
print(f"{female.name}:{seed}")
male.listen(seed, female)
male_words = male.say()



王大妈:中午好啊!我的黄金西瓜不见了，你见过吗


In [12]:
openai.api_key = "sk-8p38chfjXbbL1RT943B051229a224a8cBdE1B53b5e2c04E2"
openai.api_base = "http://api.ai-yyds.com/v1"

def query(prompt, inputs):
    content = prompt + inputs
    output = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
      messages=[{"role": "system", "content": content}, {"role": "user", "content": content}])
    output = output["choices"][0]["message"]["content"]
    return output
query("12", "李大爷")

'我不知道你指的是哪一位李大爷，可以提供更多上下文信息吗？'

In [10]:
import os
import openai

openai.api_key = "sk-8p38chfjXbbL1RT943B051229a224a8cBdE1B53b5e2c04E2"
openai.api_base = "https://api.ai-yyds.com/v1"


# 获取模型列表
def get_model_ist(str:str) -> dict:
  res = openai.Model.list(str)
  for data in res["data"]:
    print("root:{}".format(data['root']))

  return res

if __name__ =='__main__':
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "你是一个开发者"},
      {"role": "user", "content": "Hello!"}
    ]
  )

  print(completion.choices[0].message)

  # 测试使用
  get_model_ist(openai.api_key)



{
  "role": "assistant",
  "content": "Hello there! How can I assist you with your development needs?"
}
root:gpt-3.5-turbo
root:gpt-3.5-turbo-0301
root:gpt-3.5-turbo-0613
root:gpt-3.5-turbo-16k
root:gpt-3.5-turbo-16k-0613
root:gpt-4
root:gpt-4-0314
root:gpt-4-0613
root:gpt-4-32k
root:gpt-4-32k-0314
root:gpt-4-32k-0613
root:text-embedding-ada-002
root:text-davinci-003
root:text-davinci-002
root:text-curie-001
root:text-babbage-001
root:text-ada-001
root:text-moderation-latest
root:text-moderation-stable
